In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!pip install psaw
import pandas as pd
import shutil as sh
import os
import datetime
from datetime import datetime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
os.chdir('/content/drive/MyDrive/Projects/reddit-vote-predictor/')

In [4]:
import scripts.RedditScrape as rs

In [5]:
cat = rs.RedditScraper('cat', datetime(2020, 1, 1), datetime(2021, 1, 1))
cat.scrape_posts()
cat.posts.to_csv('data/tbl/cat_posts.csv')
dog = rs.RedditScraper('dog', datetime(2020, 1, 1), datetime(2021, 1, 1))
dog.scrape_posts()
dog.posts.to_csv('data/tbl/dog_posts.csv')

/usr/local/lib/python3.7/dist-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
/usr/local/lib/python3.7/dist-packages/psaw/PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
/usr/local/lib/python3.7/dist-packages/psaw/PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")
